# Sentiment Pipeline auf allen Daten

In [2]:
import pandas as pd
from transformers import pipeline
from tqdm.auto import tqdm

# 1) Daten laden (hast du schon gemacht)
df = pd.read_parquet("parlamint_at_clean.parquet")

# 2) TQDM für pandas aktivieren
tqdm.pandas(desc="Berechne Sentiment")

# 3) Lange Texte kappen (optional, aber sinnvoll für BERT)
def clip(s, n=1000):
    return s[:n] if isinstance(s, str) else s

# 4) Rows ohne Text rausfiltern (Safety)
df = df.dropna(subset=["text"]).copy()

# 5) Sentiment-Pipeline laden
# device=-1 -> CPU, device=0 -> erste GPU falls vorhanden
sent = pipeline(
    "sentiment-analysis",
    model="oliverguhr/german-sentiment-bert",
    device=-1
)

# 6) Sentiment für alle Texte berechnen – mit Fortschrittsbalken
df["sent_raw"] = df["text"].progress_apply(lambda t: sent(clip(t))[0])

# 7) Felder aufsplitten: Label, Score, numerischer Wert
df["sent_label"] = df["sent_raw"].apply(lambda x: x["label"])   # negative / neutral / positive
df["sent_score"] = df["sent_raw"].apply(lambda x: x["score"])   # Konfidenz

label2num = {"negative": -1, "neutral": 0, "positive": 1}
df["sent_value"] = df["sent_label"].map(label2num)

# 8) Ergebnis speichern
out_path = "parlamint_at_with_sentiment.parquet"
df.to_parquet(out_path, index=False)
print(f"Fertig! Mit Sentiment gespeichert als: {out_path}")
print("Zeilen:", len(df))
print(df[["text", "sent_label", "sent_score", "sent_value"]].head())


Device set to use cpu


Berechne Sentiment:   0%|          | 0/231759 [00:00<?, ?it/s]

Fertig! Mit Sentiment gespeichert als: parlamint_at_with_sentiment.parquet
Zeilen: 231759
                                                text sent_label  sent_score  \
0  Ich eröffne die 12. Sitzung des Nationalrates....    neutral    0.993581   
1  Hinsichtlich der eingelangten Verhandlungsgege...    neutral    0.999271   
2  Herr Abgeordneter Schieder, zur Geschäftsbehan...    neutral    0.988012   
3  Herr Präsident! Es steht hier in dieser schrif...    neutral    0.982424   
4  Herr Abgeordneter! Ich habe Ihre Unterlage im ...   negative    0.983540   

   sent_value  
0           0  
1           0  
2           0  
3           0  
4          -1  
